In [8]:
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd

In [9]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
data = pd.read_json(df[0].to_json(orient='records'))

In [10]:
data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
data = data[data['Borough'] != 'Not assigned']

In [12]:
#when Neighbourhood is empty, same as Borough
data['Neighbourhood'] = np.where(data['Neighbourhood'] == 'Not assigned'] , data['Borough'], data['Neighbourhood']) 


SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (<ipython-input-12-f5878045825b>, line 2)

In [13]:
#More than one neighborhood join by comma

data = data.groupby(['Borough', 'Postal Code'], as_index=False).agg(','.join)
data.shape

(103, 3)

In [14]:
url = "https://cocl.us/Geospatial_data"
geo_data = pd.read_csv(url)

In [15]:
df_pos = pd.merge(data, geo_data, on=['Postal Code'], how='inner')

In [20]:
df_pos.head()
df_tor = df_pos[['Postal Code', 'Borough', 'Neighbourhood','Latitude', 'Longitude']].copy()

df_tor.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
